In [2]:
import pandas as pd
from deep_translator import GoogleTranslator
from spacy.lang.en import English
from spacy.lang.sv import Swedish
import os

nlp_en = English()
nlp_sv = Swedish()

In [26]:
# I suc core är datan uppdelad i meningar redan (all conll?), så det behövs inte göras.
df = pd.read_pickle('data/coref.pkl')
texts = df['Text']
all_tokens_sv = []
all_tokens_en = []


In [14]:
with open('data/alignments/parallel_small.txt', 'w') as writer:
    for tokens_sv in texts:
        sentence = ' '.join(tokens_sv)
        translated_sent = GoogleTranslator(source='sv', target='en').translate(text=sentence)

        # spacy_doc_sv = nlp_sv(sent_sv.replace('\\n', ' '))
        # tokens_sv = map(lambda token: token.text, spacy_doc_sv)

        spacy_doc_en = nlp_en(translated_sent.replace('\\n', ' '))
        tokens_en = list(map(lambda token: token.text, spacy_doc_en))
        all_tokens_sv.append(tokens_sv)
        all_tokens_en.append(tokens_en)

        if translated_sent.strip() and sentence.strip():
            writer.write(sentence)
            writer.write(' ||| ')
            writer.write(' '.join(tokens_en))
            writer.write('\\n')

In [28]:
for tokens_sv in texts:
    sentence = ' '.join(tokens_sv)
    translated_sent = GoogleTranslator(source='sv', target='en').translate(text=sentence)
    # spacy_doc_sv = nlp_sv(sent_sv.replace('\\n', ' '))
    # tokens_sv = map(lambda token: token.text, spacy_doc_sv)
    spacy_doc_en = nlp_en(translated_sent.replace('\n', ' '))
    tokens_en = list(map(lambda token: token.text, spacy_doc_en))
    all_tokens_sv.append(tokens_sv)
    all_tokens_en.append(tokens_en)
print(all_tokens_en)


[['It', 'was', 'a', 'disgusting', 'journey', ',', 'but', 'since', 'the', 'worst', 'is', 'yet', 'to', 'come', ',', 'I', 'will', 'be', 'brief', '.'], ['We', 'traveled', 'among', 'Indians', 'and', 'they', 'prevented', 'us', 'from', 'making', 'our', 'tea', '.'], ['When', 'we', 'asked', 'to', 'borrow', 'a', 'pot', ',', 'they', 'were', 'furious', '.'], ['We', 'were', 'not', 'clean', 'enough', 'to', 'touch', 'their', 'pans', '.'], ['Two', 'storms', 'attacked', 'us', ',', 'some', 'died', 'and', 'were', 'thrown', 'overboard', ',', 'some', 'were', 'born', '.'], ['In', 'Alexandria', 'we', 'had', 'to', 'change', 'boats', '.'], ['There', 'were', 'many', 'Jews', 'on', 'it', ',', 'suddenly', 'we', 'were', 'surrounded', 'by', 'skulls', 'and', 'beards', 'and', 'I', 'shouted', 'to', 'Yakov', ':'], ['-', 'Look', ',', 'we', "'ve", 'done', 'it', '.'], ['Soon', 'we', 'will', 'be', 'home', '.'], ['And', 'I', 'lifted', 'him', 'up', 'and', 'spun', 'him', 'around', 'and', 'carried', 'me', 'like', 'crazy', '.'],

In [ ]:
"align_command = '../eflomal/align.py -i data/alignments/parallel.txt --priors data/alignments/sv-en2.priors --model 3 \\ -f data/alignments/suc.fwd -r data/alignments/suc.rev'"
'../eflomal/align.py -i data/alignments/parallel_small.txt --priors data/alignments/sv-en2.priors --model 3 \\ -f data/alignments/suc_small.fwd -r data/alignments/suc_small.rev'"
"sym_command = '../word-alignment/python-test/fast_align/build/atools -c grow-diag-final-and -i data/alignments/suc.fwd -j data/alignments/suc.rev >data/alignments/suc.sym'"

In [29]:
# Gör om alignments till en lista av listor med "alignment pairs"
# [0, 1] är ett alignment pair där token 0 i originaltexten ska alignas med
# token 1 i den översatta textern
formatted_alignments = []

with open('data/alignments/suc.sym', 'r') as file:
    sentence_alignments = file.read().splitlines()

    for alignment in sentence_alignments:
        list_alignment = [list(map(int, pair.split('-'))) for pair in alignment.split()]
        formatted_alignments.append(list_alignment)
    print(formatted_alignments)

[[[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [11, 14], [12, 15], [13, 16], [13, 17], [14, 16], [15, 18], [16, 19]], [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [9, 9], [11, 10], [12, 11], [13, 12]], [[0, 0], [1, 1], [2, 2], [3, 3], [5, 4], [6, 5], [7, 6], [8, 7], [9, 8], [10, 10], [11, 11]], [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [8, 7], [9, 8], [10, 9]], [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7], [8, 8], [8, 9], [9, 9], [10, 10], [11, 11], [12, 12], [13, 13], [13, 14], [14, 15]], [[0, 0], [1, 1], [2, 3], [3, 2], [4, 5], [5, 6], [6, 7]], [[0, 4], [1, 5], [2, 0], [3, 1], [4, 2], [5, 3], [6, 6], [7, 7], [8, 9], [9, 8], [10, 10], [11, 11], [12, 12], [13, 13], [14, 14], [15, 15], [16, 16], [17, 17], [18, 18], [19, 19], [20, 20]], [[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5], [6, 6], [7, 7]], [[0, 0], [1, 1], [1, 2], [2, 1], [2, 3], [3, 4], [4, 5]], [[0, 0], [1, 

In [22]:
# kvar från förr, kanske ej nödvändigt
alignment_dicts = []
for align_list in formatted_alignments:
    alignment_dict = {}
    for alignment in align_list:
        index_en = alignment[1]
        index_sv = alignment[0]
        alignment_dict[index_en] = index_sv
    alignment_dicts.append(alignment_dict)
print(alignment_dicts)


[{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 14: 11, 15: 12, 16: 14, 17: 13, 18: 15, 19: 16}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 11, 11: 12, 12: 13}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 10: 10, 11: 11}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8, 8: 9, 9: 10}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 13, 15: 14}, {0: 0, 1: 1, 3: 2, 2: 3, 5: 4, 6: 5, 7: 6}, {4: 0, 5: 1, 0: 2, 1: 3, 2: 4, 3: 5, 6: 6, 7: 7, 9: 8, 8: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}, {0: 0, 1: 2, 2: 1, 3: 2, 4: 3, 5: 4}, {0: 0, 1: 1, 2: 2, 4: 3, 3: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 13, 13: 15, 14: 16}, {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 12, 11: 13, 12: 14, 13: 15, 14: 16, 15: 17, 16: 18, 17: 19, 18: 20, 19: 21, 20: 22, 21: 23,

In [41]:
cluster_lists = df['Coref_Clusters']
new_cluster_lists = []
for clusters, alignments in zip(cluster_lists, alignment_dicts):
    new_clusters = []
    for cluster in clusters:
        new_cluster = []
        for span in cluster:  # vill egentligen kolla om något av indexen emellan finns också
            if len(span) == 1:  # snabbfix, ändra spans i dataframe sen ändra detta
                start_index = span[0]
                end_index = span[0]
            else:
                [start_index, end_index] = span
            index_span = range(start_index, end_index + 1)
            new_indices = []
            for index in index_span:
                if index in alignments:
                    new_indices.append(alignments[index])
            if new_indices:
                new_cluster.append([min(new_indices), max(new_indices)])
            # if start_index in alignment_dict:
            #     if end_index in alignment_dict:
            #         new_cluster.append([alignment_dict[start_index], alignment_dict[end_index]])
            #     else:
            #         new_cluster.append([alignment_dict[start_index], alignment_dict[start_index]])
        new_clusters.append(new_cluster)
    new_cluster_lists.append(new_clusters)
print(new_cluster_lists)

[[[]], [[[1, 1], [8, 8], [13, 13]], [[4, 4], [6, 6]]], [[[2, 2]], [[10, 10]]], [[[1, 1]], [[10, 10]]], [[[4, 4]]], [[], [[5, 5]]], [[[4, 4]], [[10, 10]], [[17, 17]], [[20, 20]]], [[[4, 4]]], [[[2, 2]]], [[[2, 2], [13, 13]], [[5, 5], [8, 8]]], [[[1, 3], [17, 17], [21, 21], [26, 26]], [[13, 13], [29, 29], [35, 35]]], [[[2, 2]]], [[[1, 1]], [[14, 14], [25, 25]], [[25, 26], [29, 29]]], [[]], [[[5, 5], [7, 7]], [], [[10, 10]]], [[[6, 6]], [[9, 9], [14, 14], [25, 25]], [[29, 31]]], [[[2, 2], [7, 7]], [], [[14, 14]], [[19, 19]], [[21, 25]]], [[[1, 1]], [[3, 3]], [[10, 10]]], [[]], [[[1, 1], [5, 5]], [[3, 3]]], [[[5, 5]], [[7, 7]]], [[[1, 1], [5, 5], [9, 9]], [[12, 12], [22, 22]], [[3, 3]]], [[[2, 2], [5, 5]], [[7, 7], [8, 8]], [[20, 20]]], [[[3, 3], [1, 1]], []], [], [[[16, 16]]], [[[1, 1]], [[4, 4]]], [[[2, 2]], [[10, 10]]], [[[2, 2]], [[6, 6]], [[8, 8]]], [[[2, 6]]], [[[7, 7]]], [], [], [], [[[2, 2]]], [[[2, 2]], [[2, 3]]], [[[1, 1]]], [], [[[14, 14], [10, 10]], [[12, 12]]], [[], [[1, 1]]],

In [54]:
#Printa ord i samma cluster
for t, c in zip(all_tokens_en, new_cluster_lists):
    for cluster in c:
        print('\n')
        for span in cluster:
            if len(span) == 1: # snabbfix, ändra spans i dataframe sen ändra detta
                i_s = span[0]
                i_e = span[0]
            else:
                [i_s, i_e] = span
            print(t[i_s-1:i_e])





['We']
['us']
['.']


['Indians']
['they']


['we']


['were']


['We']


['.']


['us']




['to']


['Jews']


['were']


['I']


['Yakov']


['we']


['we']


['I']
['like']


['up']
['him']


['A', 'large', 'Jew']
['him']
['but']
['with']


['his']
[',']
[]


['I']


['I']


['had']
['years']


['years', 'of']
['our']




['I']
['I']




['Jakov']


['me']


[',']
['defense']
['awake']


['the', 'real', 'Jerusalem']


['I']
['I']




['old']


['had']


['visa', 'to', 'Palestine', ',', 'I']


['He']


['it']


['us']




['I']
['myself']


['it']


['Jakov']


['I']


['I']
['eyes']
['disappointment']


['knew']
['in']


['Jakov']


['I']
['him']


['he']
['sat']


['the']


['longer']
['He']




['day']


['I']


['cheek']


['I']


['he']


['We']


[',']


['brother']


['the', 'old', 'man', 'also', 'tore']


['.']


['I']


['My']


['My', 'brother']


['He']


[',']
['faces']


['the']




['The']


['can']


['He']
['up']
['had']
['belongings']


['us']


['anyone']


['h

In [ ]:
# Printa ord som alignats med varandra
# verkar vara lite fel i sym-filerna, för att jag fixade fel i suc-core-mod?
for tok_sv, tok_en, aligns in zip(all_tokens_sv, all_tokens_en, formatted_alignments):
    for align_pair in aligns:
        try:
            print(tok_sv[align_pair[0]], '\t', tok_en[align_pair[1]])
        except:
            print("Error in alignment:", align_pair, tok_sv, tok_en)